In [1]:
from lxml import etree as et
import os
import re

In [2]:
os.chdir(os.path.expanduser("~/work/m-labs/artiq-hardware/AD9154_reg"))

In [3]:
chip = "ad9144".lower()
descs = set()
with open("{}_reg.h".format(chip), "w") as out:
    print("#ifndef __{}_REG_H".format(chip.upper()), file=out)
    print("#define __{}_REG_H".format(chip.upper()), file=out)
    print(file=out)
    for reg in et.parse("{}.xml".format(chip)).iter("Register"):
        addr = int(reg.find("Address").text, 16)
        desc = reg.find("Description").text.strip().upper()
        name = reg.find("Name").text.strip()
        desc = desc.upper()
        assert desc not in descs, desc
        descs.add(desc)
        width = int(reg.find("Width").text)
        assert width == 8
        assert reg.find("Exists").text == "True"
        print("#define {}_{: <32} {:#05x}".format(chip.upper(), desc, addr), file=out)
        bits = reg.findall("BitFields/BitField")
        for bit in bits:
            bit_name = bit.find("Name").text.strip()
            bit_desc = bit.find("Description").text.strip().upper()
            bit_desc = re.sub(r"\[(\d+):(\d+)\]$", r"_\1_\2", bit_desc, 1)
            bit_desc = bit_desc.replace(" ", "_")
            bit_desc = bit_desc.replace("-", "_")    
            bit_desc = bit_desc.replace("+", "")
            bit_width = int(bit.find("Width").text)
            #if len(bits) == 1 or bit_width == width:
            bit_desc += "_"
            #if bit_desc == desc:
            #    bit_desc += "_"
            assert bit_desc not in descs, bit_desc
            descs.add(bit_desc)
            assert int(bit.find("SliceWidth").text) == bit_width
            access = bit.find("Access").text.strip()
            assert int(bit.find("BitOffset").text) == 0
            default = int(bit.find("DefaultValue").text)
            offset = int(bit.find("RegOffset").text)
            notes = (bit.find("Notes").text or "").strip()
            #print("#define {}_{: <32} (1 << {})  /* {}, {:#04x} {} */".format(
            #        chip.upper(), bit_desc, offset, bit_width, default, access),
            #     file=out)
            print("/* {}: {} */".format(bit_name, notes), file=out)
            print("/* default: {:#04x}, access: {} */".format(default, access), file=out)
            if "W" in access.upper():
                print("#define {}_{: <32} (((x) & {:#x}) << {})".format(
                        chip.upper(), bit_desc + "SET(x)", (1 << bit_width) - 1, offset),
                     file=out)
            if "R" in access.upper():
                print("#define {}_{: <32} (((x) >> {}) & {:#x})".format(
                        chip.upper(), bit_desc + "GET(x)", offset, (1 << bit_width) - 1),
                     file=out)
        print(file=out)
    print("#endif", file=out)

In [11]:
chip = "ad9144".lower()
descs = set()
with open("{}_reg.tsv".format(chip), "w") as out:
    for reg in et.parse("{}.xml".format(chip)).iter("Register"):
        addr = int(reg.find("Address").text, 16)
        desc = reg.find("Description").text.strip().upper()
        name = reg.find("Name").text.strip()
        desc = desc.upper()
        assert desc not in descs, desc
        descs.add(desc)
        width = int(reg.find("Width").text)
        assert width == 8
        assert reg.find("Exists").text == "True"
        bits = reg.findall("BitFields/BitField")
        print("{:#05x}\t{}".format(addr, desc), file=out)
        for bit in bits:
            bit_name = bit.find("Name").text.strip()
            bit_desc = bit.find("Description").text.strip().upper()
            bit_desc = re.sub(r"\[(\d+):(\d+)\]$", r"_\1_\2", bit_desc, 1)
            bit_desc = bit_desc.replace(" ", "_")
            bit_desc = bit_desc.replace("-", "_")    
            bit_desc = bit_desc.replace("+", "")
            bit_width = int(bit.find("Width").text)
            #if len(bits) == 1 or bit_width == width:
            #bit_desc += "_"
            if bit_desc == desc:
                bit_desc += "_"
            assert bit_desc not in descs, bit_desc
            descs.add(bit_desc)
            assert int(bit.find("SliceWidth").text) == bit_width
            access = bit.find("Access").text.strip()
            assert int(bit.find("BitOffset").text) == 0
            default = int(bit.find("DefaultValue").text)
            offset = int(bit.find("RegOffset").text)
            notes = (bit.find("Notes").text or "").strip()
            if offset == 0 and len(bits) == 1:
                continue
            print("\t\t{}\t{}\t{:#04x}\t{}\t{}".format(
                offset, bit_width, default, access, bit_desc), file=out)
            #print("#define {}_{: <32} (1 << {})  /* {}, {:#04x} {} */".format(
            #        chip.upper(), bit_desc, offset, bit_width, default, access),
            #     file=out)
            # print("/* default: {:#04x}, access: {} */".format(default, access), file=out)
            #if "W" in access.upper():
            #    print("#define {}_{: <32} (((x) & {:#x}) << {})".format(
            #            chip.upper(), bit_desc + "SET(x)", (1 << bit_width) - 1, offset),
            #         file=out)
            #if "R" in access.upper():
            #    print("#define {}_{: <32} (((x) >> {}) & {:#x})".format(
            #            chip.upper(), bit_desc + "GET(x)", offset, (1 << bit_width) - 1),
            #         file=out)

In [1]:
chip = "ad9154".lower()
descs = set()
with open("{}_reg.tsv".format(chip), "r") as reg, open("{}_reg.h".format(chip), "w") as out:
    print("#ifndef __{}_REG_H".format(chip.upper()), file=out)
    print("#define __{}_REG_H".format(chip.upper()), file=out)
    for line in reg:
        line = line.split("\t")
        if len(line) == 2:
            addr, desc = line
            addr = int(addr, 16)
            desc = desc.strip()
            print("", file=out)
            assert desc not in descs, desc
            descs.add(desc)
            print("#define {}_{: <32} {:#05x}".format(chip.upper(), desc, addr), file=out)
        elif len(line) == 7:
            offset, bit_width, default, access, bit_desc = line[2:]
            offset = int(offset)
            bit_width = int(bit_width)
            default = int(default, 16)
            bit_desc = bit_desc.strip()
            assert bit_desc not in descs, bit_desc
            descs.add(bit_desc)
            bit_desc += "_"
            print("/* default: {:#04x}, access: {} */".format(default, access), file=out)
            if "W" in access.upper():
                print("#define {}_{: <32} (((x) & {:#x}) << {})".format(
                        chip.upper(), bit_desc + "SET(x)", (1 << bit_width) - 1, offset),
                     file=out)
            if "R" in access.upper():
                print("#define {}_{: <32} (((x) >> {}) & {:#x})".format(
                        chip.upper(), bit_desc + "GET(x)", offset, (1 << bit_width) - 1),
                     file=out)
        else:
            raise ValueError(line)
    print("#endif", file=out)

In [6]:
chip = "ad9154".lower()
descs = set()
with open("{}_reg.tsv".format(chip), "r") as reg, open("{}_reg.py".format(chip), "w") as out:
    print("# auto-generated, do not edit", file=out)
    print("from artiq.language.core import portable", file=out)
    print("from artiq.language.types import TInt32", file=out)
    for line in reg:
        line = line.split("\t")
        if len(line) == 2:
            addr, desc = line
            addr = int(addr, 16)
            desc = desc.strip()
            print("", file=out)
            assert desc not in descs, desc
            descs.add(desc)
            print("{}_{: <32} {:#05x}".format(chip.upper(), desc + " =", addr), file=out)
        elif len(line) == 7:
            offset, bit_width, default, access, bit_desc = line[2:]
            offset = int(offset)
            bit_width = int(bit_width)
            default = int(default, 16)
            bit_desc = bit_desc.strip()
            assert bit_desc not in descs, bit_desc
            descs.add(bit_desc)
            print("# default: {:#04x}, access: {}".format(default, access), file=out)
            if "W" in access.upper():
                print("@portable", file=out)
                print("def {}_{}_SET(x: TInt32) -> TInt32:\n    return (x & {:#x}) << {}\n".format(
                        chip.upper(), bit_desc, (1 << bit_width) - 1, offset),
                     file=out)
            if "R" in access.upper():
                print("@portable", file=out)
                print("def {}_{}_GET(x: TInt32) -> TInt32:\n    return (x >> {}) & {:#x}\n".format(
                        chip.upper(), bit_desc, offset, (1 << bit_width) - 1),
                     file=out)
        else:
            raise ValueError(line)